# Libraries

In [1]:
try:
    import pandas as pd
    import numpy as np
    import zipfile
    import pandasql as ps
    print("Successfully imported Libraries")
except ImportError as IE:
    print("Error importing module: {}".format(IE))
    exit(1)

Successfully imported Libraries


In [2]:
def sql(query):
    return ps.sqldf(query)

# Data

In [3]:
try:
    print("Attempting to load data ...")
    lookup_data=pd.read_csv("../../docker_intro/data/taxi+_zone_lookup.csv")
    taxi_data=pd.read_parquet("../../docker_intro/data/yellow_tripdata_2022-01.parquet",engine='pyarrow')
    print("Successfully loaded data")
except FileNotFoundError as fe:
    print("Error: {}".format(fe))
    exit(1)
except Exception as e:
    print("Error: {}".format(e))
    exit(1)

Attempting to load data ...
Successfully loaded data


In [4]:
taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
taxi_data.shape

(2463931, 19)

In [6]:
#due to the datas large size we opt to take a small sample of it and run as queries
taxi_data_sample=taxi_data.sample(n=100000)

In [7]:
#we pick 100000 random samples
taxi_data_sample.shape

(100000, 19)

In [8]:
taxi_data_sample.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
2210039,2,2022-01-28 23:16:49,2022-01-28 23:30:35,2.0,3.43,1.0,N,163,41,1,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5,0.0
1231409,2,2022-01-17 18:08:40,2022-01-17 18:12:09,1.0,0.72,1.0,N,263,263,1,4.5,0.0,0.5,2.00,0.0,0.3,9.80,2.5,0.0
1901862,2,2022-01-25 17:35:27,2022-01-25 17:41:50,2.0,0.83,1.0,N,151,238,1,6.0,1.0,0.5,2.06,0.0,0.3,12.36,2.5,0.0
1719220,2,2022-01-23 11:21:23,2022-01-23 11:27:26,1.0,1.40,1.0,N,141,236,2,7.0,0.0,0.5,0.00,0.0,0.3,10.30,2.5,0.0
1701163,2,2022-01-23 01:37:25,2022-01-23 01:41:33,1.0,0.62,1.0,N,79,148,1,5.0,0.5,0.5,1.76,0.0,0.3,10.56,2.5,0.0


In [9]:
lookup_data.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


### Merge lookup table with taxi table 

In [16]:
query='''

SELECT td.VendorID,td.PULocationID AS pickup_id,ld.Borough AS Pickup,ld.Zone AS pickup_Zone,ld2.Borough AS dropoff,ld2.Zone AS dropoff_Zone,td.fare_amount AS fare,td.passenger_count,td.trip_distance,td.tpep_pickup_datetime AS pickup_datetime,td.tpep_dropoff_datetime AS dropoff_datetime
FROM taxi_data_sample td
JOIN 
lookup_data as ld
ON td.PULocationID=ld.LocationID
JOIN
lookup_data as ld2
ON td.DOLocationID=ld2.LocationID;





'''

In [17]:
merged_data=sql(query)

In [18]:
merged_data.head()

,VendorID,pickup_id,Pickup,pickup_Zone,dropoff,dropoff_Zone,fare,passenger_count,trip_distance,pickup_datetime,dropoff_datetime
0,2,163,Manhattan,Midtown North,Manhattan,Central Harlem,13.0,2.0,3.43,2022-01-28 23:16:49.000000,2022-01-28 23:30:35.000000
1,2,263,Manhattan,Yorkville West,Manhattan,Yorkville West,4.5,1.0,0.72,2022-01-17 18:08:40.000000,2022-01-17 18:12:09.000000
2,2,151,Manhattan,Manhattan Valley,Manhattan,Upper West Side North,6.0,2.0,0.83,2022-01-25 17:35:27.000000,2022-01-25 17:41:50.000000
3,2,141,Manhattan,Lenox Hill West,Manhattan,Upper East Side North,7.0,1.0,1.40,2022-01-23 11:21:23.000000,2022-01-23 11:27:26.000000
4,2,79,Manhattan,East Village,Manhattan,Lower East Side,5.0,1.0,0.62,2022-01-23 01:37:25.000000,2022-01-23 01:41:33.000000


### Search for any missing location IDs

we search to see if there any location id records in the taxi data that aren't present in the lookup table

In [36]:
query='''

SELECT td.DOLocationID, ld.LocationID
FROM taxi_data_sample td
LEFT JOIN lookup_data as ld
ON td.DOLocationID=ld.LocationID
WHERE td.DOLocationID NOT IN (SELECT LocationID FROM lookup_data);



'''

In [37]:
missing_DO_IDs=sql(query)

In [38]:
missing_DO_IDs

,DOLocationID,LocationID


Search for any pickup IDs that are not present in the lookup table 

In [39]:
query='''

SELECT td.PULocationID, ld.LocationID
FROM taxi_data_sample td
LEFT JOIN lookup_data as ld
ON td.PULocationID=ld.LocationID
WHERE td.PULocationID NOT IN (SELECT LocationID FROM lookup_data);



'''

In [40]:
missing_PU_IDs=sql(query)

In [41]:
missing_PU_IDs

,PULocationID,LocationID


Check to see if all locations are represented in the data

In [44]:
query='''

SELECT td.PULocationID, ld.LocationID
FROM lookup_data ld
LEFT JOIN taxi_data_sample as td
ON td.PULocationID=ld.LocationID
WHERE ld.LocationID NOT IN (SELECT PULocationID FROM taxi_data_sample);



'''

In [45]:
missing_IDs=sql(query)

In [47]:
missing_IDs

,PULocationID,LocationID
0,None,2
1,None,3
2,None,6
3,None,20
4,None,27
5,None,30
6,None,46
7,None,57
8,None,58
9,None,59


This shows the areas the taxi didn't operate from

In [48]:
#areas taxis didn't drop off to 
query='''

SELECT td.DOLocationID, ld.LocationID,ld.Borough,ld.Zone
FROM lookup_data ld
LEFT JOIN taxi_data_sample as td
ON td.DOLocationID=ld.LocationID
WHERE ld.LocationID NOT IN (SELECT DOLocationID FROM taxi_data_sample);



'''

In [49]:
missing_IDs_DO=sql(query)

In [50]:
missing_IDs_DO

,DOLocationID,LocationID,Borough,Zone
0,None,2,Queens,Jamaica Bay
1,None,6,Staten Island,Arrochar/Fort Wadsworth
2,None,46,Bronx,City Island
3,None,58,Bronx,Country Club
4,None,99,Staten Island,Freshkills Park
5,None,103,Manhattan,Governor's Island/Ellis Island/Liberty Island
6,None,104,Manhattan,Governor's Island/Ellis Island/Liberty Island
7,None,105,Manhattan,Governor's Island/Ellis Island/Liberty Island
8,None,110,Staten Island,Great Kills Park
9,None,111,Brooklyn,Green-Wood Cemetery


These are the areas where no drop-offs happened

In [54]:
#get total avg fare per dropoff zone

query='''



SELECT pickup_Zone,AVG(fare) AS avg_fare
FROM merged_data
GROUP BY pickup_Zone
ORDER BY avg_fare DESC;

'''


In [55]:
avg_dropoff_fare=sql(query)

In [56]:
avg_dropoff_fare

,pickup_Zone,avg_fare
0,Highbridge Park,120.000000
1,Arden Heights,103.700000
2,Bloomfield/Emerson Hill,78.430000
3,Ocean Parkway South,76.840000
4,Marble Hill,70.200000
...,...,...
221,Brighton Beach,7.826667
222,Saint Michaels Cemetery/Woodside,5.163333
223,Queensbridge/Ravenswood,4.819595
224,Bay Terrace/Fort Totten,2.500000


### Average trip distance

In [57]:
query= '''

SELECT AVG(trip_distance)
FROM merged_data

'''

In [58]:
avg_trip_distance=sql(query)

In [59]:
avg_trip_distance

,AVG(trip_distance)
0,6.816703


### Distance traveled per day

In [80]:
merged_data['pickup_datetime']=merged_data['pickup_datetime'].astype('datetime64[ns]')

In [87]:
query= '''
SELECT CAST(pickup_datetime AS DATE) AS date,COUNT(trip_distance) AS distance
FROM merged_data
GROUP BY CAST(pickup_datetime AS DATE)
ORDER BY distance DESC;
'''

In [88]:
sql(query)

,date,distance
0,2022,99998
1,2021,1
2,2009,1


In [73]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   VendorID          100000 non-null  int64  
 1   pickup_id         100000 non-null  int64  
 2   Pickup            100000 non-null  object 
 3   pickup_Zone       99715 non-null   object 
 4   dropoff           100000 non-null  object 
 5   dropoff_Zone      99667 non-null   object 
 6   fare              100000 non-null  float64
 7   passenger_count   97113 non-null   float64
 8   trip_distance     100000 non-null  float64
 9   pickup_datetime   100000 non-null  object 
 10  dropoff_datetime  100000 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 8.4+ MB
